# Практическое задание

# Работа с эмбеддингами

## курс "Анализ неструктурированных данных"


## Введение

В этом задании вы научитесь использовать векторные представления слов для выделения слов, схожих по смыслу

### Обязательные библиотеки:
- [Gensim](https://radimrehurek.com/gensim/) — инструмент для решения различных задач NLP (тематическое моделирование, представление текстов, ...);
- [Numpy](http://www.numpy.org) — библиотека для векторных вычислений;
Для выполнения дополнительных заданий (по желанию)
- [Pandas](https://pandas.pydata.org) - библиотека для анализа табличных данных;
- [scikit-learn](http://scikit-learn.org/stable/index.html) — библилиотека с алгоритмами классического машинного обучения;
- [matplotlib](https://matplotlib.org) - библиотека для построения графиков;

-------

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

# Константы
DATA_DIR = 'data'
RANDOM_STATE = 42

### Модель, генерирующая вектора слов

Для решения вам потребуются предобученная модель векторных представлений слов.

Будем использовать русскоязычную [модель эмбеддингов](https://rusvectores.org/ru/models/) (Если функция ниже не работает - [прямая ссылка на скачивание](http://vectors.nlpl.eu/repository/20/213.zip)), обученную на GeoWAC - мультиязычном корпусе интернет-страниц.
Скачаем ее (внимание! должен быть загружен модуль utils.py - как загружать см. readme)

In [2]:
# from utils import download_model

# download_model(data_dir=DATA_DIR)

## Часть 1. Предобученные векторные представления слов

Скачайте предобученные вектора и загрузите их с помощью функции [fasttext.FastTextKeyedVectors.load](https://radimrehurek.com/gensim/models/fasttext.html) библиотеки Gensim.

In [3]:
import gensim

wv_embeddings = gensim.models.fasttext.FastTextKeyedVectors.load(
    DATA_DIR + "/model/model.model"
)

В результате в переменной `wv_embeddings` лежит FastText модель.
Это улучшенная версия Word2Vec моделей, отличающаяся в том, что модель работает с подсловами (character n-grams), а не целыми словами. Это дает несколько важных преимуществ:

1. Модель способна работать с неизвестными словами, которых не было в обучающей выборке
2. Лучше справляется со словами с опечатками
3. Эффективна для языков с богатой морфологией (включая русский)

В результате с помощью нее можно генерировать эмбеддинги произвольных слов, которые иначе были бы out of vocabulary.

Посмотрим, как "выглядит" модель: распечатаем эмбеддинг чего угодно.
На вход модели можно подавать как известные слова, так и любые неизвестные (любые сочетания символов), эмбеддинг будет построен.

In [4]:
wv_embeddings['студент']

array([-0.12469783,  0.18736716,  0.22868113,  0.27407894,  0.05341857,
        0.1317426 , -0.33944902,  0.15010178,  0.11839966, -0.05448272,
       -0.04013713,  0.19393396,  0.05723746, -0.28904027,  0.29126135,
       -0.03872602,  0.05806026, -0.1647555 ,  0.2963717 , -0.2940837 ,
       -0.01341213,  0.06515279, -0.28850263,  0.18423478,  0.12815852,
        0.10678071,  0.28139436,  0.00773022, -0.11348739,  0.14297695,
       -0.26492628, -0.129977  ,  0.07842775, -0.02894384, -0.20529068,
       -0.21005282, -0.30868313, -0.0292949 , -0.07670937, -0.26449117,
       -0.3972074 , -0.17982467,  0.13768162, -0.00637981,  0.28404418,
        0.12720445, -0.0358741 ,  0.02867263, -0.06640667,  0.04235253,
       -0.08003537, -0.13680768,  0.05537779, -0.09530751,  0.06231942,
        0.06368092, -0.08036334, -0.01808719,  0.2860434 , -0.06926312,
        0.0442103 ,  0.13214391,  0.04134564, -0.08806223,  0.05270072,
        0.19283819, -0.3059145 ,  0.41431952,  0.11099023, -0.42

Учтите, что эти эмбеддинги не нормированы (так как получаются суммированием биграмм) и для корректной работы функций их желательно нормировать. Это можно сделать вот так:

In [5]:
emb = np.array(wv_embeddings['студент'])
emb /= np.linalg.norm(emb)
emb

array([-0.03489639,  0.05243425,  0.06399587,  0.07670033,  0.01494906,
        0.03686785, -0.09499399,  0.04200562,  0.03313386, -0.01524686,
       -0.01123228,  0.05427196,  0.01601776, -0.08088722,  0.08150879,
       -0.01083738,  0.01624802, -0.04610643,  0.08293891, -0.08229862,
       -0.00375335,  0.01823285, -0.08073677,  0.05155766,  0.03586486,
        0.02988232,  0.07874753,  0.00216328, -0.03175917,  0.04001176,
       -0.07413898, -0.03637375,  0.02194782, -0.00809986, -0.0574501 ,
       -0.05878277, -0.08638423, -0.00819811, -0.02146693, -0.07401722,
       -0.11115754, -0.0503235 ,  0.03852987, -0.00178538,  0.07948908,
        0.03559786, -0.01003928,  0.00802397, -0.01858375,  0.01185225,
       -0.0223977 , -0.0382853 ,  0.01549734, -0.02667158,  0.01743994,
        0.01782095, -0.02248949, -0.00506166,  0.08004856, -0.01938312,
        0.01237215,  0.03698016,  0.01157048, -0.024644  ,  0.01474817,
        0.0539653 , -0.08560944,  0.11594632,  0.03106035, -0.11

### Методы для работы с этими векторами, встроенные в библиотеку Gensim
Все методы используют векторную арифметику: получают вектор и выбирают слова с близкими векторами. Термин "семантическая близость" здесь следует понимать не иначе как "близкое расстояние между векторами"

### **most_similar_to_given:**

Метод находит из заданного списка слово, наиболее  близкое к целевому слову.

Принимает два параметра:
- word - Целевое слово для сравнения (str)
- list_words - Список слов-кандидатов для сравнения (list of str)

Возвращает строку - слово из списка с наибольшей косинусной близостью к целевому слову.

Пример: найти с помощью данного метода наиболее похожее на слово музыка слово из списка ['вода', 'звук', 'рюкзак', 'мышь'].



In [6]:
wv_embeddings.most_similar_to_given('музыка', ['вода', 'звук', 'рюкзак', 'мышь'] )

'звук'

### **doesnt_match:**

Метод находит "белую ворону" - слово, которое  наименее похоже на остальные в представленном списке.

Принимает единственный параметр:

    words - Список слов для сравнения (list of str)

Возвращает строку - слово, которое наименее похоже на остальные слова из списка.

Пример: найти с помощью данного метода слово, которое является лишним в списке:'завтрак', 'обед', 'ужин', 'студент'

In [7]:
wv_embeddings.doesnt_match(['завтрак', 'обед', 'ужин', 'студент'])

'студент'

## **most_similar:**

Метод, находящий слова, наиболее близкие к заданным положительным примерам и далекие от отрицательных.

Принимает 2 параметра-списка (списки могут быть разной длины, непустые):

    positive - Список из строк, векторы которых будут считаться "позитивами" при поиске
    negative - Список из строк, векторы которых будут считаться "негативами" при поиске

Возвращаее список из пар (слово, косинусная близость), отсортированных по убыванию близости

Пример: найти с помощью данного метода слово, похожее на слова из названия нашего факультета и непохожее на названия других наук.

In [8]:
wv_embeddings.most_similar([ 'вычисления', 'математика','кибернетика', 'информатика'],['физика','химия', 'биология', ])

[('вычисление', 0.5770097374916077),
 ('вычислительный', 0.5413917303085327),
 ('-вычислительный', 0.5393710732460022),
 ('вычислитель', 0.5220285654067993),
 ('кибернетик', 0.4932546019554138),
 ('алгоритмизация', 0.4776235818862915),
 ('информационный', 0.47564974427223206),
 ('геоинформационный', 0.47121182084083557),
 ('кибербезопасность', 0.4706329107284546),
 ('вычислить', 0.4694153666496277)]

In [9]:
vec1 = wv_embeddings['собака']
vec2 = wv_embeddings['кошка']

cos_similarity = lambda a, b: np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

print(cos_similarity(vec1, vec2))

0.887803


In [10]:
vec1 = wv_embeddings['король']
vec2 = wv_embeddings['королева']
vec3 = wv_embeddings['мужчина']

res = vec1 - vec2 + vec3
words = wv_embeddings.similar_by_vector(res)
print(words)

[('мужчина', 0.7805652618408203), ('женщина', 0.6196458339691162), ('мужчинута', 0.5994272828102112), ('мужчинка', 0.5657992959022522), ('мужчинамя', 0.5580286979675293), ('женщинута', 0.5574212074279785), ('парень', 0.556357741355896), ('мужчине', 0.5528497099876404), ('-мужчина', 0.5521656274795532), ('иранец', 0.5368741154670715)]


In [11]:
with open('words_small.txt', encoding='utf8') as f:
    words = f.read().replace('\n', ' ').split(' ')

wv_embeddings.most_similar_to_given('автомобиль', words)

'дворник'

In [12]:
with open('words_big.txt', encoding='utf8') as f:
    words = f.read().replace('\n', ' ').split(' ')

wv_embeddings.most_similar_to_given('лом', words)

'металл'

In [23]:
many_meaning_word = wv_embeddings.most_similar( 'россия', topn=200)
many_meaning_word

[('россиия', 0.8126454949378967),
 ('росссия', 0.7835801839828491),
 ('росия', 0.7800871133804321),
 ('белоруссия', 0.7433123588562012),
 ('российский', 0.7344349026679993),
 ('казахстан', 0.733062207698822),
 ('украина', 0.7327169179916382),
 ('россий', 0.7265834808349609),
 ('сербия', 0.7168248891830444),
 ('турция', 0.7159420251846313),
 ('прибалтика', 0.6989014148712158),
 ('китай', 0.6846498847007751),
 ('россии', 0.6834197044372559),
 ('беларуссия', 0.6812626123428345),
 ('россие', 0.675833523273468),
 ('молдавия', 0.6756800413131714),
 ('франция', 0.6741426587104797),
 ('рф', 0.6703830361366272),
 ('германия', 0.6677007675170898),
 ('армения', 0.6666843891143799),
 ('иран', 0.6659193634986877),
 ('грузия', 0.6648346185684204),
 ('туркменистан', 0.658734142780304),
 ('кыргызстан', 0.6560449600219727),
 ('финляндия', 0.6556289792060852),
 ('беларусь', 0.6551189422607422),
 ('америка', 0.6537857055664062),
 ('молдова', 0.6533205509185791),
 ('индия', 0.6529909372329712),
 ('москва'

In [14]:
words = [word for word, _ in many_meaning_word]
for 

SyntaxError: invalid syntax (478083378.py, line 2)